# Preprocessing the model: Applying Feature Engineering to the Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

## Loading the data

In [2]:
root = '../data/cleaned_cars.csv'

df = pd.read_csv(root)
df.head()

,title,brand,model,type,city,gearbox,doors,color,co2_emiss,fuel_type,...,dealer,chassis,height,length,max_speed,mixed_cons,weight,tank_vol,acc,price
0,Smart Fortwo Cabrio 52 Mhd Pure Aut.,SMART,FORTWO,sports,Alicante,Automatic,2,WHITE,99,Gasoline,...,Professional,Convertible,157,270,145,4.3,780,33,13.7,5500
1,Volkswagen Scirocco 2.0 Tsi,VOLKSWAGEN,SCIROCCO,sports,Barcelona,Manual,3,WHITE,179,Gasoline,...,Professional,Coupe,140,426,235,7.6,1373,55,7.2,10900
2,Bmw Serie 2 218da Gran Coupé,BMW,SERIE 2,sports,Cantabria,Automatic,4,OTHER,109,Diesel,...,Professional,Coupe,142,453,222,4.2,1545,42,8.5,36100
3,Bmw X4 Xdrive 20d,BMW,X4,sports,Girona,Automatic,5,WHITE,142,Diesel,...,Professional,Coupe,162,467,212,5.4,1740,67,8.0,28000
4,Dodge Viper Srt-10,DODGE,VIPER,sports,Unknown,Automatic,2,RED,488,Gasoline,...,Professional,Coupe,123,446,189,21.0,1546,70,3.9,27990


## Identifying categorical features

In [3]:
df.select_dtypes(exclude=["number"]).columns.value_counts().reset_index().drop(0, axis=1)

,index
0,warranty
1,gearbox
2,chassis
3,title
4,city
5,dealer
6,fuel_type
7,color
8,model
9,brand


In [4]:
df.select_dtypes(exclude=["number"])

,title,brand,model,type,city,gearbox,color,fuel_type,warranty,dealer,chassis
0,Smart Fortwo Cabrio 52 Mhd Pure Aut.,SMART,FORTWO,sports,Alicante,Automatic,WHITE,Gasoline,YES,Professional,Convertible
1,Volkswagen Scirocco 2.0 Tsi,VOLKSWAGEN,SCIROCCO,sports,Barcelona,Manual,WHITE,Gasoline,YES,Professional,Coupe
2,Bmw Serie 2 218da Gran Coupé,BMW,SERIE 2,sports,Cantabria,Automatic,OTHER,Diesel,YES,Professional,Coupe
3,Bmw X4 Xdrive 20d,BMW,X4,sports,Girona,Automatic,WHITE,Diesel,YES,Professional,Coupe
4,Dodge Viper Srt-10,DODGE,VIPER,sports,Unknown,Automatic,RED,Gasoline,YES,Professional,Coupe
...,...,...,...,...,...,...,...,...,...,...,...
55361,Opel Corsa 1.3cdti Selective 75,OPEL,CORSA,other,Madrid,Manual,WHITE,Diesel,YES,Professional,Sedan
55362,Mercedes Clase V 250d Largo Avantgarde 7g Tronic,MERCEDES,CLASE V,other,Unknown,Automatic,BLACK,Diesel,YES,Professional,Minivan
55363,Jeep Cherokee 2.0d Longitude 4x2 103kw,JEEP,CHEROKEE,other,Ciudad Real,Manual,GREY,Diesel,YES,Professional,Offroad
55364,Bmw X3 Xdrive 30da,BMW,X3,other,Barcelona,Automatic,WHITE,Diesel,YES,Professional,Offroad


It is needed to convert these categorical variables to numbers using a OneHotEncoder. However, title will be the exception since it is not valuable to include long text in a regression model.

In [5]:
cat_cols = df.select_dtypes(exclude=["number"]).columns.drop('title')

## Transforming Categorical into Numerical: One Hot Encoding

Using a dummifyer (`pd.get_dummies`) to transform categorical values into numerical is not the right approach for a machine learning tool. `LabelEncoder` will interpret a categorical feature i.e. `[a,b,b,c]` as `[0,1,1,2]`. This will establish a wrong numeric relationship among the categorical variable. Therefore, using a `OneHotEncoder` is the right procedure to represent a categorical variable since it will create a new category per value.  

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

### When should we use each preprocessing encoding functions?

- **LabelEncoder** - for labels(response variable) coding 1,2,3… [implies order]

- **OrdinalEncoder** - for features coding 1,2,3 … [implies order]

- **Label Binarizer** - for response variable, coding 0 & 1 [ creating multiple dummy columns]

- **OneHotEncoder** - for feature variables, coding 0 & 1 [ creating multiple dummy columns]

However, `LabelBinarizer` should ideally be used for `target` and `OneHotEncoder` should be used for `features`.

Therefore, `OneHotEncoder` will be applied to these columns: `type, gearbox, fuel_type, warranty, dealer, chassis`

In [7]:
ohe = OneHotEncoder(categories='auto')
onehot_encoder = LabelBinarizer()

In [8]:
df.select_dtypes(exclude=["number"])

,title,brand,model,type,city,gearbox,color,fuel_type,warranty,dealer,chassis
0,Smart Fortwo Cabrio 52 Mhd Pure Aut.,SMART,FORTWO,sports,Alicante,Automatic,WHITE,Gasoline,YES,Professional,Convertible
1,Volkswagen Scirocco 2.0 Tsi,VOLKSWAGEN,SCIROCCO,sports,Barcelona,Manual,WHITE,Gasoline,YES,Professional,Coupe
2,Bmw Serie 2 218da Gran Coupé,BMW,SERIE 2,sports,Cantabria,Automatic,OTHER,Diesel,YES,Professional,Coupe
3,Bmw X4 Xdrive 20d,BMW,X4,sports,Girona,Automatic,WHITE,Diesel,YES,Professional,Coupe
4,Dodge Viper Srt-10,DODGE,VIPER,sports,Unknown,Automatic,RED,Gasoline,YES,Professional,Coupe
...,...,...,...,...,...,...,...,...,...,...,...
55361,Opel Corsa 1.3cdti Selective 75,OPEL,CORSA,other,Madrid,Manual,WHITE,Diesel,YES,Professional,Sedan
55362,Mercedes Clase V 250d Largo Avantgarde 7g Tronic,MERCEDES,CLASE V,other,Unknown,Automatic,BLACK,Diesel,YES,Professional,Minivan
55363,Jeep Cherokee 2.0d Longitude 4x2 103kw,JEEP,CHEROKEE,other,Ciudad Real,Manual,GREY,Diesel,YES,Professional,Offroad
55364,Bmw X3 Xdrive 30da,BMW,X3,other,Barcelona,Automatic,WHITE,Diesel,YES,Professional,Offroad


In [9]:
np.unique(df.type, return_counts=True)

(array(['big', 'familiar', 'medium', 'minivan', 'offroad', 'other',
        'small', 'sports', 'van'], dtype=object),
 array([9670, 3022, 9807, 8227, 9539, 1649, 2729, 5263, 5460]))

In [10]:
cat_columns = ['gearbox', 'fuel_type', 'warranty', 'dealer']
n_col = len(df[cat_columns].columns)

feature_arr = ohe.fit_transform(df[cat_columns]).toarray()
feature_labels = ohe.categories_

In [11]:
# Using a dictionary to produce all the new OHE columns
feature_cols = []
for k, v in dict(zip(cat_columns, feature_labels)).items():
    for i in v:
        el = k + '_' + i
        feature_cols.append(el)

ohe_features = pd.DataFrame(feature_arr, columns=feature_cols)

ohe_features

,gearbox_Automatic,gearbox_Direct,gearbox_Manual,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
55361,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
55362,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
55363,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
55364,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [12]:
data = pd.concat([df, ohe_features], axis=1)
data.head()

,title,brand,model,type,city,gearbox,doors,color,co2_emiss,fuel_type,...,gearbox_Direct,gearbox_Manual,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional
0,Smart Fortwo Cabrio 52 Mhd Pure Aut.,SMART,FORTWO,sports,Alicante,Automatic,2,WHITE,99,Gasoline,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,Volkswagen Scirocco 2.0 Tsi,VOLKSWAGEN,SCIROCCO,sports,Barcelona,Manual,3,WHITE,179,Gasoline,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,Bmw Serie 2 218da Gran Coupé,BMW,SERIE 2,sports,Cantabria,Automatic,4,OTHER,109,Diesel,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,Bmw X4 Xdrive 20d,BMW,X4,sports,Girona,Automatic,5,WHITE,142,Diesel,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,Dodge Viper Srt-10,DODGE,VIPER,sports,Unknown,Automatic,2,RED,488,Gasoline,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [13]:
df.type.value_counts().reset_index().sort_values('index')

,index,type
1,big,9670
6,familiar,3022
0,medium,9807
3,minivan,8227
2,offroad,9539
8,other,1649
7,small,2729
5,sports,5263
4,van,5460


Let's verify if the OneHotEncoder was correctly applied:

In [14]:
np.unique(df.fuel_type)

array(['Diesel', 'Electric', 'Gasoline', 'Hybrid'], dtype=object)

In [15]:
ex_o_cols = 'fuel_type_' + np.unique(df.fuel_type)

for col in ex_o_cols:
    print('N {} = {:.0f}'.format(col, data[col].sum()))

N fuel_type_Diesel = 32968
N fuel_type_Electric = 916
N fuel_type_Gasoline = 19046
N fuel_type_Hybrid = 2436


Therefore, we can drop now the `title, gearbox, fuel_type, warranty, dealer` columns

In [16]:
data.drop(cat_columns + ['title'], axis=1, inplace=True)
data.head()

,brand,model,type,city,doors,color,co2_emiss,chassis,height,length,...,gearbox_Direct,gearbox_Manual,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional
0,SMART,FORTWO,sports,Alicante,2,WHITE,99,Convertible,157,270,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,VOLKSWAGEN,SCIROCCO,sports,Barcelona,3,WHITE,179,Coupe,140,426,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,BMW,SERIE 2,sports,Cantabria,4,OTHER,109,Coupe,142,453,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,BMW,X4,sports,Girona,5,WHITE,142,Coupe,162,467,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,DODGE,VIPER,sports,Unknown,2,RED,488,Coupe,123,446,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


---

### Applying a more scalable Encoder to categorical variables with high cardinality

`Bayesian Target Encoding` technique is an improvement over the standard `Target Encoding`, because it is trying to extract information from intra-category distribution of the target variable, while Target Encoding ignores it. The method has been proven to be really helpful on data with high cardinality categorical variables. 

`Sampling Bayesian Encoder` provides a fresh look at the Bayesian Target Encoding and opens for more opportunities of research and engineering in the area of encoding of categorical features. Instead of using the moments of the posterior distribution as the new features, it suggests to sample the posterior distribution instead, and use the sampled values as the new features. The idea is based on the observation that target encoding in general, and the Bayesian target encoding in particular, can be represented as a hierarchical model that uses weak learners to discover new features. This technique is used in **ensemble models**, such as `Random Forest` and `Gradient Boosted Trees`.

`Target Encoding` will be applied to high cardinality categories: `brand, model, city, color, type, chassis`

In [17]:
from category_encoders import TargetEncoder

In [18]:
data.select_dtypes(exclude=["number"])

,brand,model,type,city,color,chassis
0,SMART,FORTWO,sports,Alicante,WHITE,Convertible
1,VOLKSWAGEN,SCIROCCO,sports,Barcelona,WHITE,Coupe
2,BMW,SERIE 2,sports,Cantabria,OTHER,Coupe
3,BMW,X4,sports,Girona,WHITE,Coupe
4,DODGE,VIPER,sports,Unknown,RED,Coupe
...,...,...,...,...,...,...
55361,OPEL,CORSA,other,Madrid,WHITE,Sedan
55362,MERCEDES,CLASE V,other,Unknown,BLACK,Minivan
55363,JEEP,CHEROKEE,other,Ciudad Real,GREY,Offroad
55364,BMW,X3,other,Barcelona,WHITE,Offroad


Applying Target Encoding using the detailed explanation of this [Medium Article](https://medium.com/analytics-vidhya/target-encoding-vs-one-hot-encoding-with-simple-examples-276a7e7b3e64) and the approach of this [Blog post](https://brendanhasz.github.io/2019/03/04/target-encoding)

In [19]:
encoder = TargetEncoder()

In [20]:
cols_to_encode = ['brand', 'model', 'city', 'color', 'type', 'chassis']
cols_encoded = ['brand_encoded', 'model_encoded', 'city_encoded', 'color_encoded', 'type_encoded', 'chassis_encoded']

data[cols_encoded] = encoder.fit_transform(data[cols_to_encode], data.price)

/home/dsc/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [21]:
data

,brand,model,type,city,doors,color,co2_emiss,chassis,height,length,...,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional,brand_encoded,model_encoded,city_encoded,color_encoded,type_encoded,chassis_encoded
0,SMART,FORTWO,sports,Alicante,2,WHITE,99,Convertible,157,270,...,0.0,1.0,0.0,1.0,11039.980916,10171.125628,15841.032810,16998.492107,28133.811134,20602.196957
1,VOLKSWAGEN,SCIROCCO,sports,Barcelona,3,WHITE,179,Coupe,140,426,...,0.0,1.0,0.0,1.0,17051.258362,13160.993789,18049.881613,16998.492107,28133.811134,31238.629315
2,BMW,SERIE 2,sports,Cantabria,4,OTHER,109,Coupe,142,453,...,0.0,1.0,0.0,1.0,23572.093803,23508.383973,18554.493671,21242.844857,28133.811134,31238.629315
3,BMW,X4,sports,Girona,5,WHITE,142,Coupe,162,467,...,0.0,1.0,0.0,1.0,23572.093803,36412.589552,16569.441935,16998.492107,28133.811134,31238.629315
4,DODGE,VIPER,sports,Unknown,2,RED,488,Coupe,123,446,...,0.0,1.0,0.0,1.0,12506.369048,17773.464491,17306.168324,17415.598758,28133.811134,31238.629315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55361,OPEL,CORSA,other,Madrid,5,WHITE,100,Sedan,148,402,...,0.0,1.0,0.0,1.0,12845.662341,11980.345216,18944.866410,16998.492107,24125.953305,15124.335073
55362,MERCEDES,CLASE V,other,Unknown,5,BLACK,158,Minivan,188,514,...,0.0,1.0,0.0,1.0,26803.194521,40412.887500,17306.168324,19068.172062,24125.953305,14164.702259
55363,JEEP,CHEROKEE,other,Ciudad Real,5,GREY,139,Offroad,163,462,...,0.0,1.0,0.0,1.0,25865.686411,28652.349939,15706.924710,17969.471689,24125.953305,23421.348833
55364,BMW,X3,other,Barcelona,5,WHITE,149,Offroad,166,465,...,0.0,1.0,0.0,1.0,23572.093803,29654.348315,18049.881613,16998.492107,24125.953305,23421.348833


After creating the new encoded features, it is possible to see that the target encoding was done based on the price:

In [22]:
data[cols_encoded]

,brand_encoded,model_encoded,city_encoded,color_encoded,type_encoded,chassis_encoded
0,11039.980916,10171.125628,15841.032810,16998.492107,28133.811134,20602.196957
1,17051.258362,13160.993789,18049.881613,16998.492107,28133.811134,31238.629315
2,23572.093803,23508.383973,18554.493671,21242.844857,28133.811134,31238.629315
3,23572.093803,36412.589552,16569.441935,16998.492107,28133.811134,31238.629315
4,12506.369048,17773.464491,17306.168324,17415.598758,28133.811134,31238.629315
...,...,...,...,...,...,...
55361,12845.662341,11980.345216,18944.866410,16998.492107,24125.953305,15124.335073
55362,26803.194521,40412.887500,17306.168324,19068.172062,24125.953305,14164.702259
55363,25865.686411,28652.349939,15706.924710,17969.471689,24125.953305,23421.348833
55364,23572.093803,29654.348315,18049.881613,16998.492107,24125.953305,23421.348833


In [23]:
data[['brand', 'brand_encoded', 'price']]

,brand,brand_encoded,price
0,SMART,11039.980916,5500
1,VOLKSWAGEN,17051.258362,10900
2,BMW,23572.093803,36100
3,BMW,23572.093803,28000
4,DODGE,12506.369048,27990
...,...,...,...
55361,OPEL,12845.662341,7500
55362,MERCEDES,26803.194521,48500
55363,JEEP,25865.686411,15999
55364,BMW,23572.093803,18300


---

### Pandas get_dummies method

It is needed to build a module to deal with all the categorical columns and transform them into numerical.

Therefore, to apply these changes to all the columns, `pd.get_dummies` can be also used. However, for this case, I will continue with the `Encoded Features` since it is a more stablished and professional method to obtain numerical variables from categorical features.

In [24]:
df.select_dtypes(include=["object"]).drop(['title'], axis=1).columns

Index(['brand', 'model', 'type', 'city', 'gearbox', 'color', 'fuel_type',
       'warranty', 'dealer', 'chassis'],
      dtype='object')

In [25]:
cat_cols = df.select_dtypes(include=["object"]).drop(['title'], axis=1).columns
df_dummy = pd.get_dummies(df, columns=cat_cols)
df_dummy.head()

,title,doors,co2_emiss,height,length,max_speed,mixed_cons,weight,tank_vol,acc,...,chassis_Convertible,chassis_Coupe,chassis_Minivan,chassis_Offroad,chassis_Pickup,chassis_Roadster,chassis_Sedan,chassis_Stationwagon,chassis_Targa,chassis_Van
0,Smart Fortwo Cabrio 52 Mhd Pure Aut.,2,99,157,270,145,4.3,780,33,13.7,...,1,0,0,0,0,0,0,0,0,0
1,Volkswagen Scirocco 2.0 Tsi,3,179,140,426,235,7.6,1373,55,7.2,...,0,1,0,0,0,0,0,0,0,0
2,Bmw Serie 2 218da Gran Coupé,4,109,142,453,222,4.2,1545,42,8.5,...,0,1,0,0,0,0,0,0,0,0
3,Bmw X4 Xdrive 20d,5,142,162,467,212,5.4,1740,67,8.0,...,0,1,0,0,0,0,0,0,0,0
4,Dodge Viper Srt-10,2,488,123,446,189,21.0,1546,70,3.9,...,0,1,0,0,0,0,0,0,0,0


In [26]:
print("Initial number of features:", df.shape[1])
print("Final number of features:", df_dummy.shape[1])

Initial number of features: 21
Final number of features: 865


The DataFrame was transformed to get `865 features` from the initial **21 features**

In [27]:
data.head()

,brand,model,type,city,doors,color,co2_emiss,chassis,height,length,...,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional,brand_encoded,model_encoded,city_encoded,color_encoded,type_encoded,chassis_encoded
0,SMART,FORTWO,sports,Alicante,2,WHITE,99,Convertible,157,270,...,0.0,1.0,0.0,1.0,11039.980916,10171.125628,15841.032810,16998.492107,28133.811134,20602.196957
1,VOLKSWAGEN,SCIROCCO,sports,Barcelona,3,WHITE,179,Coupe,140,426,...,0.0,1.0,0.0,1.0,17051.258362,13160.993789,18049.881613,16998.492107,28133.811134,31238.629315
2,BMW,SERIE 2,sports,Cantabria,4,OTHER,109,Coupe,142,453,...,0.0,1.0,0.0,1.0,23572.093803,23508.383973,18554.493671,21242.844857,28133.811134,31238.629315
3,BMW,X4,sports,Girona,5,WHITE,142,Coupe,162,467,...,0.0,1.0,0.0,1.0,23572.093803,36412.589552,16569.441935,16998.492107,28133.811134,31238.629315
4,DODGE,VIPER,sports,Unknown,2,RED,488,Coupe,123,446,...,0.0,1.0,0.0,1.0,12506.369048,17773.464491,17306.168324,17415.598758,28133.811134,31238.629315


In [28]:
data.columns

Index(['brand', 'model', 'type', 'city', 'doors', 'color', 'co2_emiss',
       'chassis', 'height', 'length', 'max_speed', 'mixed_cons', 'weight',
       'tank_vol', 'acc', 'price', 'gearbox_Automatic', 'gearbox_Direct',
       'gearbox_Manual', 'fuel_type_Diesel', 'fuel_type_Electric',
       'fuel_type_Gasoline', 'fuel_type_Hybrid', 'warranty_NO', 'warranty_YES',
       'dealer_Individual', 'dealer_Professional', 'brand_encoded',
       'model_encoded', 'city_encoded', 'color_encoded', 'type_encoded',
       'chassis_encoded'],
      dtype='object')

Dropping the remaining categorical variables

In [29]:
data.drop(cols_to_encode, axis=1, inplace=True)
data.head()

,doors,co2_emiss,height,length,max_speed,mixed_cons,weight,tank_vol,acc,price,...,warranty_NO,warranty_YES,dealer_Individual,dealer_Professional,brand_encoded,model_encoded,city_encoded,color_encoded,type_encoded,chassis_encoded
0,2,99,157,270,145,4.3,780,33,13.7,5500,...,0.0,1.0,0.0,1.0,11039.980916,10171.125628,15841.032810,16998.492107,28133.811134,20602.196957
1,3,179,140,426,235,7.6,1373,55,7.2,10900,...,0.0,1.0,0.0,1.0,17051.258362,13160.993789,18049.881613,16998.492107,28133.811134,31238.629315
2,4,109,142,453,222,4.2,1545,42,8.5,36100,...,0.0,1.0,0.0,1.0,23572.093803,23508.383973,18554.493671,21242.844857,28133.811134,31238.629315
3,5,142,162,467,212,5.4,1740,67,8.0,28000,...,0.0,1.0,0.0,1.0,23572.093803,36412.589552,16569.441935,16998.492107,28133.811134,31238.629315
4,2,488,123,446,189,21.0,1546,70,3.9,27990,...,0.0,1.0,0.0,1.0,12506.369048,17773.464491,17306.168324,17415.598758,28133.811134,31238.629315


In [30]:
data.dtypes

doors                    int64
co2_emiss                int64
height                   int64
length                   int64
max_speed                int64
mixed_cons             float64
weight                   int64
tank_vol                 int64
acc                    float64
price                    int64
gearbox_Automatic      float64
gearbox_Direct         float64
gearbox_Manual         float64
fuel_type_Diesel       float64
fuel_type_Electric     float64
fuel_type_Gasoline     float64
fuel_type_Hybrid       float64
warranty_NO            float64
warranty_YES           float64
dealer_Individual      float64
dealer_Professional    float64
brand_encoded          float64
model_encoded          float64
city_encoded           float64
color_encoded          float64
type_encoded           float64
chassis_encoded        float64
dtype: object

All the features are now set to numerical!

Now the dataset is fully ready to apply a Machine Learning Regression Algorithm.

In [31]:
data.shape

(55366, 27)

In [32]:
output = '../data/cars_reg.csv'
data.to_csv(output, index=False)

Output generated to be used for the regression algorithms.

---

## Exploring Numerical features

On the other hand, there are more numerical columns that will be included in the regression model

In [33]:
df.select_dtypes(exclude=["object"]).columns.value_counts().reset_index().drop(0, axis=1)

,index
0,height
1,mixed_cons
2,length
3,co2_emiss
4,max_speed
5,price
6,doors
7,acc
8,tank_vol
9,weight


In [34]:
df.select_dtypes(exclude=["object"])

,doors,co2_emiss,height,length,max_speed,mixed_cons,weight,tank_vol,acc,price
0,2,99,157,270,145,4.3,780,33,13.7,5500
1,3,179,140,426,235,7.6,1373,55,7.2,10900
2,4,109,142,453,222,4.2,1545,42,8.5,36100
3,5,142,162,467,212,5.4,1740,67,8.0,28000
4,2,488,123,446,189,21.0,1546,70,3.9,27990
...,...,...,...,...,...,...,...,...,...,...
55361,5,100,148,402,164,3.8,1237,45,14.8,7500
55362,5,158,188,514,206,6.0,2145,67,9.1,48500
55363,5,139,163,462,187,5.3,1828,60,10.9,15999
55364,5,149,166,465,230,6.0,1800,67,6.2,18300


In [35]:
df.isna().sum()

title         0
brand         0
model         0
type          0
city          0
gearbox       0
doors         0
color         0
co2_emiss     0
fuel_type     0
warranty      0
dealer        0
chassis       0
height        0
length        0
max_speed     0
mixed_cons    0
weight        0
tank_vol      0
acc           0
price         0
dtype: int64